In [66]:

%pip install snowflake-connector-python
%pip install snowflake-snowpark-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [68]:

import os
import configparser
from snowflake.snowpark import Session
import snowflake.snowpark.functions as f
from snowflake.snowpark.window import Window
from datetime import date


In [67]:
config = configparser.ConfigParser()
config.read('config.ini')


['config.ini']

### creating snowpark session

In [69]:

def snowpark_session_create():
    connection_params = {
        "account": config.get("snowflake", "account"),
        "user": config.get("snowflake", "user"),
        "password": config.get("snowflake", "password"),
        "role": config.get("snowflake", "role"),
        "warehouse": config.get("snowflake", "warehouse"),
        "database": config.get("snowflake", "database"),
        "schema": config.get("snowflake", "schema")
    }

    session = Session.builder.configs(connection_params).create()
    return session


In [70]:
demo_session = snowpark_session_create()

In [71]:
df = demo_session.sql("SELECT * FROM DEMO.RAW.STG_CITIES LIMIT ")
df.show()


-----------------------------------------------------------------------------------------------------------------------------
|"CITY_ID"  |"CITY"  |"DATE"      |"TAVG"  |"TMIN"  |"TMAX"  |"PRCP"  |"SNOW"  |"WDIR"  |"WSPD"  |"WPGT"  |"PRES"  |"TSUN"  |
-----------------------------------------------------------------------------------------------------------------------------
|1          |Kyiv    |2022-08-29  |25.3    |16.6    |31.5    |0.0     |NULL    |278.0   |6.8     |27.8    |1012.5  |NULL    |
|1          |Kyiv    |2022-09-09  |13.1    |6.8     |18.3    |0.3     |NULL    |82.0    |7.0     |24.1    |1022.4  |NULL    |
|1          |Kyiv    |2022-09-08  |14.9    |9.2     |20.5    |0.0     |NULL    |15.0    |4.7     |22.2    |1019.4  |NULL    |
|1          |Kyiv    |2022-09-07  |12.4    |6.7     |19.0    |0.0     |NULL    |281.0   |4.4     |22.2    |1022.3  |NULL    |
|1          |Kyiv    |2022-09-06  |10.9    |6.8     |14.6    |0.0     |NULL    |344.0   |5.7     |25.9    |1024.7  |NU

In [ ]:
for row in df.collect():
    print(row)

### point session to the snowflake demo db,schema,table


In [72]:
demo_session.use_database("demo")
demo_session.use_schema("raw")
demo_session.table("stg_cities")

In [73]:
df = df.withColumn("DATE", f.to_date(f.col("DATE")))

In [ ]:
print(df.count())
df.show()


In [74]:
df = (
    df.select(
        "CITY",
        "DATE",
        f.col("TAVG").alias("TEMP_AVG"),
        f.col("TMAX").alias("TEMP_MAX"),
        f.col("TMIN").alias("TEMP_MIN"),
        f.col("PRCP").alias("TOTAL_PRECIPITATION"),
        f.col("WSPD").alias("AVG_WIND_SPEED"),
        f.col("WPGT").alias("WIND_PEAK_GUST"),
        f.col("PRES").alias("SEA_LEVEL_AIR_PRESSURE")
    )
    .filter(f.col("DATE").between(date(2020, 6, 1), date(2020, 8, 31)))
    .sort(f.col("DATE").asc())
)




In [ ]:
df.show(100)

In [75]:
df = df.na.fill({"TOTAL_PRECIPITATION": 0, "AVG_WIND_SPEED": 0, "WIND_PEAK_GUST": 0, "SEA_LEVEL_AIR_PRESSURE": 0})
df.show(100)


-------------------------------------------------------------------------------------------------------------------------------------------------------
|"CITY"  |"DATE"      |"TEMP_AVG"  |"TEMP_MAX"  |"TEMP_MIN"  |"TOTAL_PRECIPITATION"  |"AVG_WIND_SPEED"  |"WIND_PEAK_GUST"  |"SEA_LEVEL_AIR_PRESSURE"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------
|Oslo    |2020-06-01  |19.6        |27.2        |10.3        |0.1                    |9.9               |23.0              |1027.3                    |
|Zagreb  |2020-06-01  |18.4        |23.2        |11.8        |0.0                    |8.3               |22.2              |1016.9                    |
|Kyiv    |2020-06-01  |11.8        |16.0        |9.0         |0.0                    |16.8              |35.2              |1007.8                    |
|Munich  |2020-06-01  |16.1        |22.5        |8.3         |0.0                    |10

In [ ]:
df.show(100)

In [ ]:
window_spec = Window.partition_by(f.month(f.col("DATE"))).order_by(f.col("CITY"))


In [77]:
df = df.group_by("CITY").agg(f.avg("TEMP_AVG").alias("MONTHLY_AVERAGE_TEMP"))

In [78]:
df = df.with_column("TEMP_AVG", f.avg("MONTHLY_AVERAGE_TEMP").over(window_spec))

In [79]:
df.show()

SnowparkSQLException: (1304): 01b873dc-0001-401a-0000-ed5900049482: 000904 (42000): SQL compilation error: error line 1 at position 92
invalid identifier 'DATE'